In [2]:
def read_pdb_atoms(pdb_file):
    """
    读取PDB文件中的原子记录，并返回一个列表，包含原子名称、残基名称、残基编号、链标识以及完整的行内容。
    """
    atoms = []
    try:
        with open(pdb_file, 'r') as file:
            for line in file:
                if line.startswith("ATOM"):
                    atom_name = line[12:16].strip()
                    residue_name = line[17:20].strip()
                    residue_number = int(line[22:26].strip())
                    chain_id = line[21].strip()
                    atoms.append((atom_name, residue_name, residue_number, chain_id, line))
    except FileNotFoundError:
        print(f"Error: The file '{pdb_file}' does not exist.")
        return []
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return []
    return atoms

def reorder_pdb(source_pdb, target_pdb, output_pdb):
    """
    根据目标PDB文件的原子顺序重新排列源PDB文件的原子记录。
    """
    # 读取源PDB文件和目标PDB文件的原子信息
    source_atoms = read_pdb_atoms(source_pdb)
    target_atoms = read_pdb_atoms(target_pdb)
    
    if not source_atoms or not target_atoms:
        print("Error: One or both PDB files contain no ATOM records.")
        return
    
    # 创建一个映射，用于快速查找源PDB文件中的原子行
    source_atom_map = {(atom[0], atom[1], atom[2], atom[3]): atom[4] for atom in source_atoms}
    
    # 读取源PDB文件的全部内容
    try:
        with open(source_pdb, 'r') as file:
            source_lines = file.readlines()
    except Exception as e:
        print(f"An error occurred while reading the source PDB file: {e}")
        return
    
    # 创建一个新的PDB文件内容
    new_pdb_lines = []
    for target_atom in target_atoms:
        if target_atom[:4] in source_atom_map:
            # 如果目标原子在源文件中存在，则直接添加对应的整行内容
            new_pdb_lines.append(source_atom_map[target_atom[:4]])
        else:
            print(f"Warning: Atom {target_atom} not found in source PDB file.")
    
    # 将新的PDB内容写入输出文件
    try:
        with open(output_pdb, 'w') as file:
            file.writelines(new_pdb_lines)
        print(f"Reordered PDB file saved to {output_pdb}")
    except Exception as e:
        print(f"An error occurred while writing the output PDB file: {e}")

# 示例调用
if __name__ == "__main__":
    source_pdb = "5awl_reH.pdb"  # 源PDB文件路径
    target_pdb = "chig.pdb"  # 模板PDB文件路径
    output_pdb = "5awl_reorder.pdb"  # 输出PDB文件路径
    reorder_pdb(source_pdb, target_pdb, output_pdb)

Reordered PDB file saved to 5awl_reorder.pdb
